# Pós-graduação Lato Sensu em Ciência de Dados e Big Data - PUC MG

## TCC turma 2020 - Gabriel Fonseca da Silva


### Objetivo: desenvolver um algoritmo de machine learning para prever a aprovação de um candidato na prova do ENCCEJA 2019 para obtenção do certificado do ensino fundamental

## Dados utilizados:

### a) Microdados do ENCCEJA Nacional Regular 2019 dos inscritos que fizeram as provas do ensino fundamental

##### https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/encceja

### b) Dados de homicídios por UF do ano de 2019 extraídos do DATASUS (grupo CID10 X85-Y09 e Y35-Y36)
#### http://tabnet.datasus.gov.br/cgi/deftohtm.exe?sim/cnv/ext10uf.def

### c) População por UF estimada extraída do DATASUS (estimativas utilizadas pelo TCU)
#### http://tabnet.datasus.gov.br/cgi/tabcgi.exe?ibge/cnv/poptuf.def

### d) Índice de Desenvolvimeno Humano por Estado extraído do Atlas Brasil do IPEA
#### http://www.atlasbrasil.org.br/



## Aplicação: Importar e filtrar os dados que serão utilizados

### 1) Importando bibliotecas


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


### 2) Carregando os dados

#### 2.1) Microdados Encceja 2019

In [2]:
# Importando os dados do Encceja 2019
dados_encceja = pd.read_csv("01-dados_originais/MICRODADOS_ENCCEJA_NACIONAL_REGULAR_2019.csv", sep = ',', 
                            encoding = "ISO-8859-1")

print("\nDimensões: {0}".format(dados_encceja.shape))
print("\nCampos: {0}".format(dados_encceja.keys()))



Dimensões: (2973376, 91)

Campos: Index(['NU_INSCRICAO', 'NU_ANO', 'TP_CERTIFICACAO', 'NU_IDADE', 'TP_SEXO',
       'CO_UF_PROVA', 'SG_UF_PROVA', 'NO_ENTIDADE_CERTIFICADORA',
       'IN_PROVA_LC', 'IN_PROVA_MT', 'IN_PROVA_CN', 'IN_PROVA_CH',
       'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 'TP_PRESENCA_CN', 'TP_PRESENCA_CH',
       'CO_PROVA_LC', 'CO_PROVA_MT', 'CO_PROVA_CN', 'CO_PROVA_CH',
       'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_CN', 'NU_NOTA_CH',
       'IN_APROVADO_LC', 'IN_APROVADO_MT', 'IN_APROVADO_CN', 'IN_APROVADO_CH',
       'TX_RESPOSTAS_LC', 'TX_RESPOSTAS_MT', 'TX_RESPOSTAS_CN',
       'TX_RESPOSTAS_CH', 'TX_GABARITO_LC', 'TX_GABARITO_MT', 'TX_GABARITO_CN',
       'TX_GABARITO_CH', 'TP_STATUS_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2',
       'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO',
       'Q01', 'Q02', 'Q03', 'Q04', 'Q05', 'Q06', 'Q07', 'Q08', 'Q09', 'Q10',
       'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20',
       'Q21', 

In [3]:
# Filtrando os inscritos do ensino fundamental no Encceja 2019 que foram fazer a prova


dados_encceja_v1 = dados_encceja[(dados_encceja.IN_PROVA_MT == 1) & (dados_encceja.TP_PRESENCA_MT == 1) &
                                 (dados_encceja.IN_PROVA_LC == 1) & (dados_encceja.TP_PRESENCA_LC == 1) &
                                 (dados_encceja.IN_PROVA_CN == 1) & (dados_encceja.TP_PRESENCA_CN == 1) &
                                 (dados_encceja.IN_PROVA_CH == 1) & (dados_encceja.TP_PRESENCA_CH == 1) &                                 
                                 (dados_encceja.TP_CERTIFICACAO == 1)]


print("\nDimensões: {0}".format(dados_encceja_v1.shape))


Dimensões: (145815, 91)


In [4]:
# Selecionando as variáveis que serão inicialmente utilizados

dados_encceja_v2 = dados_encceja_v1[['NU_INSCRICAO','NU_IDADE','TP_SEXO','SG_UF_PROVA','IN_APROVADO_LC','IN_APROVADO_CN',
                                     'IN_APROVADO_CH','IN_APROVADO_MT','Q01','Q02','Q03','Q04','Q05','Q06','Q07','Q08','Q09',
                                     'Q15','Q16','Q17','Q18','Q19','Q20','Q21','Q31','Q32']]
print("\nDimensões: {0}".format(dados_encceja_v2.shape))
dados_encceja_v2.head(5)


Dimensões: (145815, 26)


NU_INSCRICAO  NU_IDADE TP_SEXO SG_UF_PROVA  IN_APROVADO_LC  \
54   180006402779      21.0       M          AM             0.0   
127  180003642665      37.0       F          SP             1.0   
290  180005144125      46.0       M          SP             1.0   
335  180005161608      46.0       F          SP             1.0   
344  180005278223      43.0       F          SP             1.0   

     IN_APROVADO_CN  IN_APROVADO_CH  IN_APROVADO_MT Q01 Q02  ... Q09 Q15 Q16  \
54              1.0             1.0             0.0   B   B  ...   K   B   D   
127             1.0             1.0             1.0   B   B  ...   K   B   B   
290             1.0             1.0             0.0   C   C  ...   D   E   B   
335             1.0             1.0             1.0   C   C  ...   I   E   A   
344             1.0             1.0             1.0   B   A  ...   D   E   C   

    Q17 Q18 Q19 Q20 Q21 Q31 Q32  
54    C   C   C   G   D   A NaN  
127   A   B   D   F   E   B NaN  
290   A   D   C   F   E   A NaN  
335   A   A   C   H   E   B NaN  
344   A   A   C   G   E   B NaN  

[5 rows x 26 columns]

#### 2.2) Dados de homicídios do DataSus

In [5]:
# Importando os dados de homicídios do DataSus

homicidios = pd.read_csv("01-dados_originais/datasus_homicidios_2019.csv", sep = ';', skiprows = [0,1,2,3,32,33,34,35,36,37,38,39,40,41,42],
                         encoding = "ISO-8859-1", engine='python')

homicidios = homicidios[['Unidade da Federação','Total']]


# Renomeando as variáveis para retirar acentos e "ç"

homicidios.rename(columns={'Unidade da Federação': 'UF'}, inplace = True)
homicidios.rename(columns={'Total': 'homicidios'}, inplace = True)


# Criando a variável SG_UF_PROVA no mesmo padrão do dataset Encceja 2019

homicidios["SG_UF_PROVA"]=homicidios["UF"]


# Recodificando a variável SG_UF_PROVA para deixá-la no mesmo padrão do dataset Encceja 2019

homicidios["SG_UF_PROVA"]=homicidios["SG_UF_PROVA"].replace(["11 Rondônia","12 Acre","13 Amazonas","14 Roraima","15 Pará",
                                                             "16 Amapá","17 Tocantins","21 Maranhão","22 Piauí","23 Ceará",
                                                             "24 Rio Grande do Norte","25 Paraíba","26 Pernambuco","27 Alagoas",
                                                             "28 Sergipe","29 Bahia","31 Minas Gerais","32 Espírito Santo",
                                                             "33 Rio de Janeiro","35 São Paulo","41 Paraná","42 Santa Catarina",
                                                             "43 Rio Grande do Sul","50 Mato Grosso do Sul","51 Mato Grosso",
                                                             "52 Goiás","53 Distrito Federal"],["RO","AC","AM","RR","PA","AP",
                                                                                                "TO","MA","PI","CE","RN","PB",
                                                                                                "PE","AL","SE","BA","MG","ES",
                                                                                                "RJ","SP","PR","SC","RS","MS",
                                                                                                "MT","GO","DF"])


# Excluindo a variável "UF"
del homicidios['UF']


# Visualizando parte dos dados

print("\nDimensões: {0}".format(homicidios.shape))
print("\nCampos: {0}".format(homicidios.keys()))
homicidios.head(5)



Dimensões: (27, 2)

Campos: Index(['homicidios', 'SG_UF_PROVA'], dtype='object')


homicidios SG_UF_PROVA
0         447          RO
1         325          AC
2        1592          AM
3         234          RR
4        3405          PA

#### 2.3) Dados de população do DataSus

In [6]:
# Importando os dados de população do DataSus

populacao = pd.read_csv("01-dados_originais/datasus_populacao_2019.csv", sep = ';', skiprows = [0,1,2,31,32,33,34,35,36,37,38,39,40,41,42,43,],
                        encoding = "ISO-8859-1", engine='python')

# Renomeando as variáveis para retirar acentos e "ç"

populacao.rename(columns={'Unidade da Federação': 'UF'}, inplace = True)
populacao.rename(columns={'População_estimada': 'Populacao'}, inplace = True)


# Criando a variável SG_UF_PROVA no mesmo padrão do dataset Encceja 2019

populacao["SG_UF_PROVA"]=populacao["UF"]


# Recodificando a variável SG_UF_PROVA para deixá-la no mesmo padrão do dataset Encceja 2019

populacao["SG_UF_PROVA"]=populacao["SG_UF_PROVA"].replace(["11 Rondônia","12 Acre","13 Amazonas","14 Roraima","15 Pará",
                                                             "16 Amapá","17 Tocantins","21 Maranhão","22 Piauí","23 Ceará",
                                                             "24 Rio Grande do Norte","25 Paraíba","26 Pernambuco","27 Alagoas",
                                                             "28 Sergipe","29 Bahia","31 Minas Gerais","32 Espírito Santo",
                                                             "33 Rio de Janeiro","35 São Paulo","41 Paraná","42 Santa Catarina",
                                                             "43 Rio Grande do Sul","50 Mato Grosso do Sul","51 Mato Grosso",
                                                             "52 Goiás","53 Distrito Federal"],["RO","AC","AM","RR","PA","AP",
                                                                                                "TO","MA","PI","CE","RN","PB",
                                                                                                "PE","AL","SE","BA","MG","ES",
                                                                                                "RJ","SP","PR","SC","RS","MS",
                                                                                                "MT","GO","DF"])




# Excluindo a variável "UF"
del populacao['UF']


# Visualizando parte dos dados

print("\nDimensões: {0}".format(populacao.shape))
print("\nCampos: {0}".format(populacao.keys()))
populacao.head(5)


Dimensões: (27, 2)

Campos: Index(['Populacao', 'SG_UF_PROVA'], dtype='object')


Populacao SG_UF_PROVA
0    1777225          RO
1     881935          AC
2    4144597          AM
3     605761          RR
4    8602865          PA

In [7]:
# Juntando os dados de homicidios com a população para calcular a taxa de homicídios

taxa_homicidios = pd.merge(homicidios,populacao, on = 'SG_UF_PROVA')
print("\nDimensões: {0}".format(taxa_homicidios.shape))
print("\nCampos: {0}".format(taxa_homicidios.keys()))
taxa_homicidios.head(5)


Dimensões: (27, 3)

Campos: Index(['homicidios', 'SG_UF_PROVA', 'Populacao'], dtype='object')


homicidios SG_UF_PROVA  Populacao
0         447          RO    1777225
1         325          AC     881935
2        1592          AM    4144597
3         234          RR     605761
4        3405          PA    8602865

In [8]:
# Calculando a taxa de homicídios por 100 mil habitantes

taxa_homicidios["taxapor100mil"]=round(100000*(taxa_homicidios.homicidios)/(taxa_homicidios.Populacao),2)

taxa_homicidios.head(5)

homicidios SG_UF_PROVA  Populacao  taxapor100mil
0         447          RO    1777225          25.15
1         325          AC     881935          36.85
2        1592          AM    4144597          38.41
3         234          RR     605761          38.63
4        3405          PA    8602865          39.58

#### 2.4) Dados do Índice de Desenvolvimento Humano do IPEA


In [9]:
# Importando os dados do IDHM

idhm = pd.read_excel("01-dados_originais/data.xlsx", skiprows = [1,29,30,31], usecols=[0, 1])


# Criando a variável SG_UF_PROVA no mesmo padrão do dataset Encceja 2019

idhm["SG_UF_PROVA"]=idhm["Territorialidades"]


# Recodificando a variável SG_UF_PROVA para deixá-la no mesmo padrão do dataset Encceja 2019

idhm["SG_UF_PROVA"]=idhm["SG_UF_PROVA"].replace(["Acre","Alagoas","Amapá","Amazonas","Bahia","Ceará","Distrito Federal",
                                                 "Espírito Santo","Goiás","Maranhão","Mato Grosso do Sul","Mato Grosso",
                                                 "Minas Gerais","Paraíba","Paraná","Pará","Pernambuco","Piauí","Rio de Janeiro",
                                                 "Rio Grande do Norte","Rio Grande do Sul","Rondônia","Roraima",
                                                 "Santa Catarina","São Paulo","Sergipe","Tocantins"],
                                                ["AC","AL","AP","AM","BA","CE","DF","ES","GO","MA","MS","MT","MG","PB","PR",
                                                 "PA","PE","PI","RJ","RN","RS","RO","RR","SC","SP","SE","TO"])


# Renomeando a variável "IDHM 2017" para retirar o espaço

idhm.rename(columns={'IDHM 2017': 'idhm_2017'}, inplace = True)


# Excluindo a variável "Territorialidades"
del idhm['Territorialidades']


# Visualizando parte dos dados

print("\nDimensões: {0}".format(idhm.shape))
print("\nCampos: {0}".format(idhm.keys()))
idhm.head(5)



Dimensões: (27, 2)

Campos: Index(['idhm_2017', 'SG_UF_PROVA'], dtype='object')


idhm_2017 SG_UF_PROVA
0      0.719          AC
1      0.683          AL
2      0.740          AP
3      0.733          AM
4      0.714          BA

#### 2.5) Enriquecimento de dados acrescentando a taxa de homicídios e o IDHM nos microdados do Encceja 2019

In [10]:
dados1 = pd.merge(dados_encceja_v2,taxa_homicidios, on = 'SG_UF_PROVA')
dados2 = pd.merge(dados1,idhm, on = 'SG_UF_PROVA')

# Visualizando parte dos dados

print("\nDimensões: {0}".format(dados2.shape))
print("\nCampos: {0}".format(dados2.keys()))
dados2.head(5)


Dimensões: (145815, 30)

Campos: Index(['NU_INSCRICAO', 'NU_IDADE', 'TP_SEXO', 'SG_UF_PROVA', 'IN_APROVADO_LC',
       'IN_APROVADO_CN', 'IN_APROVADO_CH', 'IN_APROVADO_MT', 'Q01', 'Q02',
       'Q03', 'Q04', 'Q05', 'Q06', 'Q07', 'Q08', 'Q09', 'Q15', 'Q16', 'Q17',
       'Q18', 'Q19', 'Q20', 'Q21', 'Q31', 'Q32', 'homicidios', 'Populacao',
       'taxapor100mil', 'idhm_2017'],
      dtype='object')


NU_INSCRICAO  NU_IDADE TP_SEXO SG_UF_PROVA  IN_APROVADO_LC  IN_APROVADO_CN  \
0  180006402779      21.0       M          AM             0.0             1.0   
1  180004795206      29.0       F          AM             1.0             1.0   
2  180006161956      32.0       M          AM             1.0             1.0   
3  180006088681      45.0       M          AM             1.0             1.0   
4  180005393819      22.0       F          AM             0.0             1.0   

   IN_APROVADO_CH  IN_APROVADO_MT Q01 Q02  ... Q18 Q19 Q20 Q21 Q31 Q32  \
0             1.0             0.0   B   B  ...   C   C   G   D   A NaN   
1             1.0             1.0   C   A  ...   A   A   G   E   A NaN   
2             1.0             1.0   B   A  ...   B   C   E   D   B NaN   
3             1.0             1.0   C   C  ...   B   E   F   D   B NaN   
4             0.0             1.0   B   C  ...   B   A   I   F   A NaN   

  homicidios Populacao taxapor100mil idhm_2017  
0       1592   4144597         38.41     0.733  
1       1592   4144597         38.41     0.733  
2       1592   4144597         38.41     0.733  
3       1592   4144597         38.41     0.733  
4       1592   4144597         38.41     0.733  

[5 rows x 30 columns]

In [11]:
# Salvando os dados

dados2.to_csv('02-dados_filtrados/dados_filtrados.csv',index=False)
